In [1]:
!pip install eli5

In [2]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats.stats import kendalltau
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LSTM, Masking
from tensorflow.keras.layers import MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import activations
sns.set()

In [3]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *

train_values = np.empty(shape=[0, 222])
test_values = np.empty(shape=[0, 222])

train_sampled_all = glob.glob('./drive/MyDrive/compsci/train_sampled_all_220.csv')
# test_sampled_all = glob.glob('./drive/MyDrive/compsci/test_sampled_all_centered_220.csv')
test_unsampled_all = glob.glob('./drive/MyDrive/compsci/test_unsampled_all_220.csv')

for j in train_sampled_all:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    train_values = np.append(train_values, csvrows, axis=0)
    
# for j in test_all:
#     print('Loading ', j)
#     csvrows = np.loadtxt(j, delimiter=',')
#     test_values = np.append(test_values, csvrows, axis=0)

for j in test_unsampled_all:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    test_values = np.append(test_values, csvrows, axis=0)
    
print(train_values.shape)
print(test_values.shape)

Loading  ./drive/MyDrive/compsci/train_sampled_all_220.csv
Loading  ./drive/MyDrive/compsci/test_unsampled_all_220.csv
(31912, 222)
(37863, 222)


In [4]:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]

y_train = train_values[:,-2]
y_test = test_values[:,-2]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

y_train_gc = (y_train - y_train.min())/(y_train.max()-y_train.min())*(9-1)
y_test_gc = (y_test - y_test.min())/(y_test.max()-y_test.min())*(9-1)
print(y_train_gc.shape)
print(y_test_gc.shape)

(31912, 220)
(37863, 220)
(31912,)
(37863,)
(31912,)
(37863,)


In [5]:
# # X_train = X_train.reshape(-1, X_train.shape[1],1)
# # X_test = X_test.reshape(-1, X_test.shape[1],1)

# X_train1 = X_train.reshape(X_train.shape + (1,1))
# X_test1 = X_test.reshape(X_test.shape + (1,1))
# print(X_train1.shape)
# print(X_test1.shape)

# y_train1=to_categorical(y_train)
# y_test1=to_categorical(y_test)
# print(y_train1.shape)
# print(y_test1.shape)

In [6]:
N_train = train_values
N_test = test_values[test_values[:,-2]==2]
print(N_train.shape)
print(N_test.shape)

X_train = N_train[:,:-2]
X_test = N_test[:,:-2]

y_train = train_values[:len(N_train),-2]
y_test = test_values[:len(N_test),-2]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train1 = X_train.reshape(X_train.shape + (1,1,))
X_test1 = X_test.reshape(X_test.shape + (1,1,))
print(X_train1.shape)
print(X_test1.shape)

y_train1=to_categorical(y_train)
y_test1=to_categorical(y_test)
print(y_train1.shape)
print(y_test1.shape)

(31912, 222)
(1660, 222)
(31912, 220)
(1660, 220)
(31912,)
(1660,)
(31912, 220, 1, 1)
(1660, 220, 1, 1)
(31912, 9)
(1660, 9)


In [7]:
def showResults(test, pred):
    #target_names = ['positive', 'negative']
    # print(classification_report(test, pred, target_names=target_names))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted')
    f1Score=f1_score(test, pred, average='weighted') 
    #loss=log_loss(test,pred)
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    #print("Loss : {}".format(loss))
    cm=confusion_matrix(test, pred)
    print(cm)
    return cm

In [8]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

verbose, epoch, batch_size = 1, 5, 256
activationFunction='relu'

def getlstmModel():
    
    lstmmodel = Sequential()
    lstmmodel.add(LSTM(128, return_sequences=True, input_shape=(X_train1.shape[1],1)))
    lstmmodel.add(LSTM(9, return_sequences=True))
    lstmmodel.add(MaxPooling1D(pool_size=2))
    lstmmodel.add(Flatten())
    lstmmodel.add(Dense(512, activation=tf.nn.relu))    
    lstmmodel.add(Dense(128, activation=tf.nn.relu))    
    lstmmodel.add(Dense(32, activation=tf.nn.relu))
    lstmmodel.add(Dense(9, activation='softmax'))
    lstmmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    lstmmodel.summary()
    return lstmmodel

lstmmodel = getlstmModel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 220, 128)          66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 220, 9)            4968      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 110, 9)            0         
_________________________________________________________________
flatten (Flatten)            (None, 990)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               507392    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4

In [ ]:
lstmmodelhistory= lstmmodel.fit(X_train1[:,:,:,0], y_train1, epochs=epoch, verbose=verbose, validation_split=0.2, batch_size = batch_size)

Train on 25529 samples, validate on 6383 samples
Epoch 1/5
25529/25529 [==============================] - ETA: 0s - loss: 0.6458 - accuracy: 0.7991

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


25529/25529 [==============================] - 47s 2ms/sample - loss: 0.6458 - accuracy: 0.7991 - val_loss: 0.2518 - val_accuracy: 0.9259
Epoch 2/5
25529/25529 [==============================] - 47s 2ms/sample - loss: 0.1929 - accuracy: 0.9407 - val_loss: 0.1865 - val_accuracy: 0.9414
Epoch 3/5
25529/25529 [==============================] - 46s 2ms/sample - loss: 0.1357 - accuracy: 0.9561 - val_loss: 0.1472 - val_accuracy: 0.9547
Epoch 4/5
14336/25529 [===============>..............] - ETA: 18s - loss: 0.1054 - accuracy: 0.9672

In [ ]:
############## Get CAM ################
import matplotlib.pyplot as plt
# from matplotlib.backends.backend_pdf import PdfPages

get_last_conv1 = keras.backend.function([lstmmodel.layers[0].input, keras.backend.learning_phase()], [lstmmodel.layers[1].output])
last_conv1_lstm = get_last_conv1([X_test1[:1000,:,:,0]])[0]

get_softmax1_lstm = keras.backend.function([lstmmodel.layers[0].input, keras.backend.learning_phase()], [lstmmodel.layers[-1].output])
softmax1_lstm = get_softmax1_lstm(([X_test1[:1000,:,:,0]]))[0]
softmax_weight1_lstm = lstmmodel.get_weights()[-1]
softmax_weight_lstm = np.reshape(softmax_weight1_lstm,(9,1))

CAM = np.dot(last_conv1_lstm, softmax_weight_lstm)

# pp = PdfPages('CAM.pdf')
for k in range(1):
    CAM = (CAM - CAM.min(axis=1, keepdims=True)) / (CAM.max(axis=1, keepdims=True) - CAM.min(axis=1, keepdims=True))
    c = np.exp(CAM) / np.sum(np.exp(CAM), axis=1, keepdims=True)
    plt.figure(figsize=(18, 4))
    plt.plot(X_test1[k].squeeze())
    plt.scatter(np.arange(len(X_test1[k])), X_test1[k].squeeze(), cmap='inferno_r', c=c[k,:].squeeze(), s=50)
    plt.title('True label:' + str(N_test[k,-2]) + '   likelihood of label ' + str(N_test[k,-2]) + ': ' + str(softmax1_lstm[k][int(y_test[k])]))
    # plt.title('True label:' + str(y_test[k]) + '   likelihood of label ' + str(y_test[k]) + ': ' + str(softmax1_lstm[k][int(y_test[k])]))
    plt.clim(0.003,0.010)
    plt.colorbar()
plt.show

In [ ]:
lstmpredictions = lstmmodel.predict(X_test1[:,:,:,0], verbose=1)

In [ ]:
lstm_predict=np.argmax(lstmpredictions,axis=1)
lstm_actual_value=np.argmax(y_test1,axis=1)
lstm_cf_m = showResults(lstm_actual_value, lstm_predict)
from sklearn import metrics
lstmmetrics = metrics.classification_report(lstm_actual_value, lstm_predict, digits=3)
print(lstmmetrics, 'lstm metrics')
categories=['N','L','R','V','A','F','f','/']
plt.figure(figsize=(8,6))
lstm_cf_m = lstm_cf_m.astype('float')/ lstm_cf_m.sum(axis=1)[:,np.newaxis]
sns.heatmap(lstm_cf_m,annot=True,fmt='.2%',xticklabels=categories,yticklabels=categories,vmin=0,vmax=1)
plt.title('LSTM confusion matrix')
plt.show()

In [ ]:
from itertools import islice
def means_of_slices(iterable, slice_size):
    iterator = iter(iterable)
    while True:
        slice = list(islice(iterator, slice_size))
        if slice:
            yield np.sum(slice)/len(slice)
        else:
            return
a = last_conv1_lstm
new_last_lstm = []
for i in range(len(last_conv1_lstm)):
  means = list(means_of_slices(a[i], 20))
  new_last_lstm.append(means)
new_last_lstm = np.array(new_last_lstm)
print(new_last_lstm.shape)

final_last_lstm = []
for i in new_last_lstm:
  final_last_lstm.append(np.repeat(i,20))
final_last_lstm = np.array(final_last_lstm)
print(final_last_lstm.shape)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.neural_network import MLPClassifier

NNMLP_clf1 = MLPClassifier(random_state=48, max_iter=50)
NNMLP_clf1.fit(new_last_lstm, y_test1[:1000])
perm = PermutationImportance(NNMLP_clf1).fit(new_last_lstm, y_test1[:1000])
print('LSTM result')
eli5.show_weights(perm)

In [ ]:
conv_df_lstm = pd.DataFrame(new_last_lstm)
corr_lstm = conv_df_lstm.corr(method='kendall')
slices_nums = [1,2,3,4,5,6,7,8,9,10,11]
rcParams['figure.figsize'] = 8,6
sns.heatmap(corr_lstm,annot=True,xticklabels=slices_nums,yticklabels=slices_nums,vmin=-0.6,vmax=1,fmt='0.1g')
plt.title('LSTM Kendall Tau Corellation')
plt.show()

In [ ]:
############## Get CAM ################
import matplotlib.pyplot as plt

CAM = np.dot(last_conv1_lstm, softmax_weight_lstm)

# pp = PdfPages('CAM.pdf')
for k in range(20):
    CAM = (CAM - CAM.min(axis=1, keepdims=True)) / (CAM.max(axis=1, keepdims=True) - CAM.min(axis=1, keepdims=True))
    c = np.exp(CAM) / np.sum(np.exp(CAM), axis=1, keepdims=True)
    plt.figure(figsize=(18, 4))
    plt.plot(X_test1[k].squeeze())
    plt.scatter(np.arange(len(X_test1[k])), X_test1[k].squeeze(), cmap='inferno_r', c=c[k,:].squeeze(), s=50)
    plt.plot(final_last_lstm[k], color = 'darkcyan')
    plt.title('True label:' + str(N_test[k,-2]) + '   likelihood of label ' + str(N_test[k,-2]) + ': ' + str(softmax1_lstm[k][int(y_test[k])]))
    # plt.title('True label:' + str(y_test[k]) + '   likelihood of label ' + str(y_test[k]) + ': ' + str(softmax1_lstm[k][int(y_test[k])]))
    plt.clim(0.003,0.010)
    plt.colorbar()
plt.show

In [ ]:
# # saving the model

# model_json = lstmmodel.to_json()
# with open("NEWlstmmodel_testunsam.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# lstmmodel.save_weights("NEWlstmmodel_testunsam_weights.h5")
# lstmmodel.save("NEWlstmmodel_testunsam.h5")
# print("Saved model to disk")

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

verbose, epoch, batch_size = 1, 5, 64
activationFunction='relu'

def getModel():
    
    cnnmodel = Sequential()
    cnnmodel.add(Conv1D(filters=128, kernel_size=16,padding='same', activation='relu',input_shape=(X_train1.shape[1],1)))
    cnnmodel.add(BatchNormalization())
    cnnmodel.add(Conv1D(filters=32, kernel_size=16,padding='same', activation='relu'))
    cnnmodel.add(BatchNormalization())
    cnnmodel.add(Conv1D(filters=9, kernel_size=16,padding='same', activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2,padding='same'))
    cnnmodel.add(Flatten())
    cnnmodel.add(Dense(512, activation='relu'))
    cnnmodel.add(Dense(128, activation='relu'))
    cnnmodel.add(Dense(32, activation='relu'))
    cnnmodel.add(Dense(9, activation='softmax'))
    cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    cnnmodel.summary()
    return cnnmodel

cnnmodel = getModel()

In [ ]:
modelhistory= cnnmodel.fit(X_train1[:,:,:,0], y_train1, epochs=epoch, verbose=verbose, validation_split=0.2, batch_size = batch_size)

In [ ]:
############## Get CAM ################
import matplotlib.pyplot as plt
# from matplotlib.backends.backend_pdf import PdfPages

get_last_conv1 = keras.backend.function([cnnmodel.layers[0].input, keras.backend.learning_phase()], [cnnmodel.layers[4].output])
last_conv1_cnn = get_last_conv1([X_test1[:1000,:,:,0]])[0]

get_softmax1_cnn = keras.backend.function([cnnmodel.layers[0].input, keras.backend.learning_phase()], [cnnmodel.layers[-1].output])
softmax1_cnn = get_softmax1_cnn(([X_test1[:1000,:,:,0]]))[0]
softmax_weight1_cnn = cnnmodel.get_weights()[-1]
softmax_weight_cnn = np.reshape(softmax_weight1_cnn,(9,1))

CAM = np.dot(last_conv1_cnn, softmax_weight_cnn)

# pp = PdfPages('CAM.pdf')
for k in range(1):
    CAM = (CAM - CAM.min(axis=1, keepdims=True)) / (CAM.max(axis=1, keepdims=True) - CAM.min(axis=1, keepdims=True))
    c = np.exp(CAM) / np.sum(np.exp(CAM), axis=1, keepdims=True)
    plt.figure(figsize=(18, 4))
    plt.plot(X_test1[k].squeeze())
    plt.scatter(np.arange(len(X_test1[k])), X_test1[k].squeeze(), cmap='inferno_r', c=c[k,:].squeeze(), s=50)
    plt.title('True label:' + str(N_test[k,-2]) + '   likelihood of label ' + str(N_test[k,-2]) + ': ' + str(softmax1_cnn[k][int(y_test[k])]))
    # plt.title('True label:' + str(y_test[k]) + '   likelihood of label ' + str(y_test[k]) + ': ' + str(softmax1_cnn[k][int(y_test[k])]))
    plt.clim(0.003,0.010)
    plt.colorbar()
plt.show

In [ ]:
cnnpredictions = cnnmodel.predict(X_test1[:,:,:,0], verbose=1)

In [ ]:
cnn_predict=np.argmax(cnnpredictions,axis=1)
cnn_actual_value=np.argmax(y_test1,axis=1)
CNN_cf_m = showResults(cnn_actual_value, cnn_predict)
from sklearn import metrics
metrics = metrics.classification_report(cnn_actual_value, cnn_predict, digits=3)
print(metrics, 'CNN metrics')
categories=['N','L','R','V','A','F','f','/']
plt.figure(figsize=(8,6))
CNN_cf_m = CNN_cf_m.astype('float')/ CNN_cf_m.sum(axis=1)[:,np.newaxis]
sns.heatmap(CNN_cf_m,annot=True,fmt='.2%',xticklabels=categories,yticklabels=categories,vmin=0,vmax=1)
plt.title('1D-CNN confusion matrix')
plt.show()

In [ ]:
from itertools import islice
def means_of_slices(iterable, slice_size):
    iterator = iter(iterable)
    while True:
        slice = list(islice(iterator, slice_size))
        if slice:
            yield np.sum(slice)/len(slice)
        else:
            return
a = last_conv1_cnn
new_last_conv1 = []

for i in range(len(last_conv1_cnn)):
  means = list(means_of_slices(a[i], 20))
  new_last_conv1.append(means)
new_last_conv1 = np.array(new_last_conv1)
print(new_last_conv1.shape)

final_last_conv1 = []
for i in new_last_conv1:
  final_last_conv1.append(np.repeat(i,20))
final_last_conv1 = np.array(final_last_conv1)
print(final_last_conv1.shape)

In [ ]:
import eli5
from eli5 import format_as_image
from eli5.sklearn import PermutationImportance
from sklearn.neural_network import MLPClassifier
NNMLP_clf = MLPClassifier(random_state=48, max_iter=50)
NNMLP_clf.fit(new_last_conv1, y_test1[:1000])
perm = PermutationImportance(NNMLP_clf).fit(new_last_conv1, y_test1[:1000])
print('CNN results')
eli5.show_weights(perm)

In [ ]:
conv_df_slices = pd.DataFrame(new_last_conv1)
corr_slices = conv_df_slices.corr(method='kendall')
slices_nums = [1,2,3,4,5,6,7,8,9,10,11]
rcParams['figure.figsize'] = 8,6
sns.heatmap(corr_slices,annot=True,xticklabels=slices_nums,yticklabels=slices_nums,vmin=-0.6,vmax=1,fmt='0.1g')
plt.title('CNN Kendall Tau Corellation')
plt.show()

In [ ]:
############## Get CAM ################
import matplotlib.pyplot as plt

CAM = np.dot(last_conv1_cnn, softmax_weight_cnn)

# pp = PdfPages('CAM.pdf')
for k in range(1):
    CAM = (CAM - CAM.min(axis=1, keepdims=True)) / (CAM.max(axis=1, keepdims=True) - CAM.min(axis=1, keepdims=True))
    c = np.exp(CAM) / np.sum(np.exp(CAM), axis=1, keepdims=True)
    plt.figure(figsize=(18, 4))
    plt.plot(X_test1[k].squeeze())
    plt.scatter(np.arange(len(X_test1[k])), X_test1[k].squeeze(), cmap='inferno_r', c=c[k,:].squeeze(), s=50)
    plt.plot(final_last_conv1[k], color = 'darkcyan')
    plt.title('True label:' + str(N_test[k,-2]) + '   likelihood of label ' + str(N_test[k,-2]) + ': ' + str(softmax1_cnn[k][int(y_test[k])]))
    # plt.title('True label:' + str(y_test[k]) + '   likelihood of label ' + str(y_test[k]) + ': ' + str(softmax1_cnn[k][int(y_test[k])]))
    plt.clim(0.003,0.010)
    plt.colorbar()
plt.show

In [ ]:
from sklearn.preprocessing import normalize

n1= normalize(final_last_conv1)
n2= normalize(final_last_lstm)
n3= normalize(last_conv1_cnn[:,:,0])
n4= normalize(last_conv1_lstm[:,:,0])
n5= normalize(X_test)

m1=n1.mean(axis=0)
m2=n2.mean(axis=0)
m3=n3.mean(axis=0)
m4=n4.mean(axis=0)


plt.figure(figsize=(12,4))
plt.plot(n5[0],color='b')
plt.plot(n5[0],color='b',label='ECG Beat')
plt.plot(m1,color='g',label='Conv1D layer')
plt.plot(m2,color='r',label='LSTM layer')
plt.title('Class: N, Number: 1')
plt.legend()
plt.show()

plt.figure(figsize=(12,4))
plt.plot(n5[0],color='b')
plt.plot(n5[0],color='b',label='ECG Beat')
plt.plot(m1*2,color='g',label='Conv1D layer')
plt.plot(m2*2,color='r',label='LSTM layer')
plt.title('Class: N, Number: 1')
plt.legend()
plt.show()

In [ ]:
# # saving the model

# model_json = cnnmodel.to_json()
# with open("NEWcnnmodel_testunsam.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# cnnmodel.save_weights("NEWcnnmodel_testunsam_weights.h5")
# cnnmodel.save("NEWcnnmodel_testunsam.h5")
# print("Saved model to disk")

In [ ]:
# from sklearn.decomposition import PCA
# x_pca = PCA(n_components=50,random_state=42).fit_transform(X_train)

In [ ]:
# plt.figure(figsize=(8,8))

# categories=['N','L','R','V','A','F','f','/']
# scatter = plt.scatter(x_pca[:,0],x_pca[:,1],c=y_train, cmap='inferno')
# plt.legend(title="Classes",loc='upper right',*scatter.legend_elements())
# plt.title('PCA')

In [ ]:
# from sklearn.manifold import TSNE
# x_tsne = TSNE(n_components=2,random_state=42,perplexity=100, verbose=5).fit_transform(x_pca)

In [ ]:
# plt.figure(figsize=(8,8))

# categories=['N','L','R','V','A','F','f','/']
# scatter = plt.scatter(x_tsne[:,0],x_tsne[:,1],c=y_train, cmap='inferno')
# plt.legend(title="Classes",loc='upper right',*scatter.legend_elements())
# plt.title('PCA and TSNE')

In [ ]:
# import umap
# reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=100, a = 0.5, b = 1.2)
# embedding = reducer.fit_transform(x_pca)
# embedding.shape

In [ ]:
# plt.figure(figsize=(8,8))

# plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], c=y_train, cmap='inferno')
# plt.legend(title="Classes",loc='upper right',*scatter.legend_elements())
# plt.title('PCA and UMAP')

In [ ]:
# from keras.models import model_from_json
# #loading the model and running it on datasets
# from keras.models import load_model

# saved_model = load_model('/content/drive/My Drive/compsci/modelsresult/NEWcnnmodel_testsam.h5')
# saved_model.summary()
# json_file = open('/content/drive/My Drive/compsci/modelsresult/NEWcnnmodel_testsam.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("/content/drive/My Drive/compsci/modelsresult/NEWcnnmodel_testsam_weights.h5")
# print("Loaded model from disk")

# loaded_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
# score = loaded_model.evaluate(X_test1[:,:,:,0], y_test1, verbose=1)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
 # # Plot training & validation loss values
# plt.plot(modelhistory.history['loss'])
# plt.plot(modelhistory.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.savefig('Loss.png', format='png', dpi=1200)
# plt.show()


# # Plot training & validation accuracy values
# plt.plot(modelhistory.history['accuracy'])
# plt.plot(modelhistory.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='lower right')
# plt.savefig('Accuracy.png', format='png', dpi=1200)
# plt.show()

In [ ]:
# from sklearn import metrics
# y_pred_CNN = loaded_model.predict(X_test1[:,:,:,0])
# y_pred_CNN = np.argmax(y_pred_CNN, axis=1)
# y_test = np.argmax(y_test1, axis=1)
# print(y_pred_CNN)
# metrics = metrics.classification_report(y_test, y_pred_CNN, digits=3)
# print(metrics, 'CNN metrics')

In [ ]:
# CNN_cf_m = showResults(y_test, y_pred_CNN)
# from sklearn import metrics
# metrics = metrics.classification_report(y_test, y_pred_CNN, digits=3)
# print(metrics, 'CNN metrics')
# categories=['N','L','R','V','A','F','f','/']
# plt.figure(figsize=(8,6))
# CNN_cf_m = CNN_cf_m.astype('float')/ CNN_cf_m.sum(axis=1)[:,np.newaxis]
# sns.heatmap(CNN_cf_m,annot=True,fmt='.2%',xticklabels=categories,yticklabels=categories,vmin=0,vmax=1)
# plt.title('1D-CNN confusion matrix')
# plt.show()

In [ ]:
# N_train = train_values[train_values[:,-2]==7]
# train_df = pd.DataFrame(N_train[:,:-1])

In [ ]:
# for i in range(20):
#   sns.set()
#   plt.figure(figsize=(6,2))
#   plt.plot(train_df.iloc[i,:186])
#   label = train_df.iloc[i,187]
#   print(label)
#   plt.title('Class: f, Number: 7')
#   plt.show()